Notebook created in compliance with Hamoye assessment requirement...

# Import Modules

In [1]:
import numpy as np
import pandas as pd

# Data Exploration

In [26]:
df = pd.read_csv('energydata_complete.csv')

df.shape

(19735, 29)

In [18]:
df.columns

Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [19]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [27]:
reduced_df = df.drop(['date','lights'], 1)

# Scale and Normalize data

In [32]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

norm_df = pd.DataFrame(scaler.fit_transform(reduced_df), columns=reduced_df.columns)

features_df = norm_df.drop('Appliances', 1)
target = norm_df['Appliances']

features_df.head()

# Data Split
- 70 - 30 train-test split
- random_state = 42

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_df, target, test_size=0.3, random_state=42)

# Train a Linear Model

In [35]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

linear_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [38]:
# make prediction
predictions = linear_model.predict(X_test)

# Evaluation
- MAE
- RSE
- RMSE
- R2

In [45]:
# Evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, predictions)
print('mae value = ',round(mae, 2))

rse = np.sum(np.square(y_test - predictions))
print('rse value = ',round(rse, 2))

rmse = np.sqrt(mean_squared_error(y_test, predictions))
print('rmse value = ',round(rmse, 3))

r2 = r2_score(y_test, predictions)
print('r2/ coeff of determination value = ',round(r2, 2))

mae value =  0.05
rse value =  45.35
rmse value =  0.088
r2/ coeff of determination value =  0.15


Function to return weights of features

In [52]:
def get_feature_weights(model, feat, col_name):

  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)

  return weights_df

Feature Weights

In [53]:
linear_weights = get_feature_weights(linear_model, X_train, 'Linear_Model_weights')
print(linear_weights)

       Features  Linear_Model_weights
0          RH_2             -0.456698
1         T_out             -0.321860
2            T2             -0.236178
3            T9             -0.189941
4          RH_8             -0.157595
5        RH_out             -0.077671
6          RH_7             -0.044614
7          RH_9             -0.039800
8            T5             -0.015657
9            T1             -0.003281
10          rv1              0.000770
11          rv2              0.000770
12  Press_mm_hg              0.006839
13           T7              0.010319
14   Visibility              0.012307
15         RH_5              0.016006
16         RH_4              0.026386
17           T4              0.028981
18    Windspeed              0.029183
19         RH_6              0.038049
20         RH_3              0.096048
21           T8              0.101995
22    Tdewpoint              0.117758
23           T6              0.236425
24           T3              0.290627
25         R

# Ridge Model

In [55]:
from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha=0.4)

ridge_model.fit(X_train, y_train)

Ridge(alpha=0.4, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [58]:
ridge_predictions = ridge_model.predict(X_test)

rmse_ridge = np.sqrt(mean_squared_error(y_test, ridge_predictions))
print('rmse_ridge value = ',round(rmse_ridge, 3))

rmse_ridge value =  0.088


# Lasso Model

In [59]:
from sklearn.linear_model import Lasso
lasso_model = Lasso(alpha=0.001)

lasso_model.fit(X_train, y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [60]:
lasso_weights = get_feature_weights(lasso_model, X_train, 'Lasso_Model_weights')
print(lasso_weights)

       Features  Lasso_Model_weights
0        RH_out            -0.049557
1          RH_8            -0.000110
2            T1             0.000000
3     Tdewpoint             0.000000
4    Visibility             0.000000
5   Press_mm_hg            -0.000000
6         T_out             0.000000
7          RH_9            -0.000000
8            T9            -0.000000
9            T8             0.000000
10         RH_7            -0.000000
11          rv1            -0.000000
12           T7            -0.000000
13           T6             0.000000
14         RH_5             0.000000
15           T5            -0.000000
16         RH_4             0.000000
17           T4            -0.000000
18         RH_3             0.000000
19           T3             0.000000
20         RH_2            -0.000000
21           T2             0.000000
22         RH_6            -0.000000
23          rv2            -0.000000
24    Windspeed             0.002912
25         RH_1             0.017880


In [61]:
lasso_predictions = lasso_model.predict(X_test)

rmse_lasso = np.sqrt(mean_squared_error(y_test, lasso_predictions))
print('rmse_lasso value = ',round(rmse_lasso, 3))

rmse_lasso value =  0.094


Question 12
- From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two D.P?

In [39]:
from sklearn.metrics import r2_score
r2 = r2_score(df['T2'], df['T6'])
print('r2 value = ',round(r2, 2))

r2 value =  -35.39
